In [1]:
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install seaborn
%pip install plotly
%pip install sklearn
%pip install xboost
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement xboost (from versions: none)
ERROR: No matching distribution found for xboost


Note: you may need to restart the kernel to use updated packages.


In [2]:
#Previsão de máquinas que irão falhar usando o método de regressão

#Importando bibliotecas
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import plotly.express as px
sns.set_style('white', { 'axes.spines.right': False, 'axes.spines.top': False})
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support as score, roc_curve
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from sklearn.utils import compute_sample_weight
from xgboost import XGBClassifier
import xgboost as xgb
import xgboost

# Carregando os dados de treino
path = '/home/fernanda/Downloads/Desafio Ciências de Dados/'
df = pd.read_csv(path + "desafio_manutencao_preditiva_treino.csv") 

# Definindo o alvo
target_name='failure_type'

# Print do resumo dos dados de treino(10 primeiros)
print(df.shape[0])
df.head(10)

ModuleNotFoundError: No module named 'sklearn'

**# Limpando os dados**

In [ ]:
# Procura por valores faltantes
def mss_values(df):
    null_df = pd.DataFrame(df.isna().sum(), columns=['null_values']).sort_values(['null_values'], ascending=False)
    fig = plt.subplots(figsize=(16, 6))
    ax = sns.barplot(data=null_df, x='null_values', y=null_df.index, color='royalblue')
    pct_values = [' {:g}'.format(elm) + ' ({:.1%})'.format(elm/len(df)) for elm in list(null_df['null_values'])]
    ax.set_title('Visão geral dos valores ausentes')
    ax.bar_label(container=ax.containers[0], labels=pct_values, size=12)

if df.isna().sum().sum() > 0:
    mss_values(df)
else:
    print('sem valores ausentes')

# Descarte de todas as colunas com mais de 5% de valores ausentes
for col_name in df.columns:
    if df[col_name].isna().sum()/df.shape[0] > 0.05:
        df.drop(columns=[col_name], inplace=True) 

df.columns

In [ ]:
# Retirando a coluna 'product_id' e 'id'
df_base = df.drop(columns=['product_id', 'udi'])

# Ajustando o nome das colunas
df_base.rename(columns={'Air temperature [K]': 'air_temperature', 
                        'Process temperature [K]': 'process_temperature', 
                        'Rotational speed [rpm]':'rotational_speed', 
                        'Torque [Nm]': 'torque', 
                        'Tool wear [min]': 'tool_wear'}, inplace=True)
df_base.head()

 **# Explorando os dados**

In [ ]:
# Exibição da distribuição de classe da variável de destino

px.histogram(df_base, y="failure_type", color="failure_type") 

In [ ]:
# Mapa de dispersão (pairplot) nos tipos de falhas
sns.pairplot(df_base, height=2.5, hue='failure_type')

In [ ]:
# Mapa de correlação
plt.figure(figsize=(6,4))
sns.heatmap(df_base.corr(), cbar=True, fmt='.1f', vmax=0.8, annot=True, cmap='Blues')

In [ ]:
# Criação do histogramas para explicitar os tipos de falhas
def histogram(column_name):
    plt.figure(figsize=(16,6))
    return px.box(data_frame=df_base, y=column_name, color='failure_type', points="all", width=1200)

histogram('air_temperature_k')

In [ ]:
histogram('process_temperature_k')

In [ ]:
histogram('rotational_speed_rpm')

In [ ]:
histogram('torque_nm')

In [ ]:
histogram('tool_wear_min')

**# Preparação dos dados**

In [ ]:
def preparacao_dados(df_base, target_name):
    df = df_base.dropna()

    df['target_name_encoded'] = df[target_name].replace({'No Failure': 0, 'Power Failure': 1, 'Tool Wear Failure': 2, 'Overstrain Failure': 3, 'Random Failures': 4, 'Heat Dissipation Failure': 5})
    df['type'].replace({'L': 0, 'M': 1, 'H': 2}, inplace=True)
    X = df.drop(columns=[target_name, 'target_name_encoded'])
    y = df['target_name_encoded'] #Prediction label

    # Dividindo os dados em conjuntos de dados x_train e y_train
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)

    # Printando 'O resultado é:' (rows, training_sequence, features) (prediction value, )
    print('train: ', X_train.shape, y_train.shape)
    print('test: ', X_test.shape, y_test.shape)
    return X, y, X_train, X_test, y_train, y_test

# Removendo o alvo (Falilure Type) dos dados de treino e tornando ele nos dados de teste
X, y, X_train, X_test, y_train, y_test = preparacao_dados(df_base, target_name)

**# Treinando o modelo**

In [ ]:
weight_train = compute_sample_weight('balanced', y_train)
weight_test = compute_sample_weight('balanced', y_test)

xgb_model = xgboost.XGBClassifier(eta=0.1, max_depth=8, 
                                    colsample_bytree=0.5, 
                                    booster='gbtree', 
                                    sampling_method='gradient_based', 
                                    eval_metric='aucpr', 
                                    objective='multi:softmax', 
                                    tree_method='hist', 
                                    num_class=6)
                                    
# Ajustando o modelo aos dados
xgb_model.fit(X_train, y_train.ravel(), sample_weight=weight_train)

# **Avaliação do Modelo**

In [ ]:
# Contagem do modelo com o conjunto de dados teste
score = xgb_model.score(X_test, y_test.ravel(), sample_weight=weight_test)

# Prevendo o conjunto de dados de teste
y_pred = xgb_model.predict(X_test)

# Print da classificação e acurácia
results_log = classification_report(y_test, y_pred)
print(results_log)

In [ ]:
# validação cruzada
val_cruzada  = cross_validate(xgb_model, X_train, y_train, cv=10, scoring="f1_weighted", fit_params={ "sample_weight" :weight_train})
val_cruzada_df = pd.DataFrame(val_cruzada)
px.bar(x=val_cruzada_df.index, y=val_cruzada_df.test_score, width=800)

In [ ]:
# Criando as previsões no conjunto de dados de teste
y_pred = xgb_model.predict(X_test)

# Print da matriz de multiclasse
matriz = confusion_matrix(y_test, y_pred)
df_matriz = pd.DataFrame(matriz, columns=np.unique(y_test), index=np.unique(y_test))
df_matriz.index.name = 'Actual'
df_matriz.columns.name = 'Predicted'
plt.figure(figsize = (8, 5))
sns.set(font_scale=1.1) #for label size
sns.heatmap(df_matriz, cbar=True, cmap= "inferno", annot=True, fmt='.0f')

In [ ]:
df_matriz.to_csv('previsao.csv')